In [1]:
import re
import pandas as pd
from pathlib import Path
from typing import Iterable, Optional, Union, List

def search_workbook(
    xlsx_path: Union[str, Path],
    queries: Union[str, Iterable[str]],
    columns: Optional[Iterable[str]] = None,    # search only these columns (None = all)
    whole_word: bool = True,                    # match as whole word (e.g., LDHA not “oldham”)
    case_sensitive: bool = False,               # LDHA == ldha if False
    use_regex: bool = False,                    # supply your own regex in `queries`
    return_first_only: bool = False,            # stop at first match per sheet
) -> pd.DataFrame:
    """
    Scan every sheet of an Excel file for `queries`.
    Return a DataFrame of all matching rows, with extra columns:
      _sheet: sheet name
      _row   : original (0-based) row index in that sheet
      _query : which query matched
      _match_cols: comma-separated columns that matched in that row
    """
    xlsx_path = Path(xlsx_path)
    if isinstance(queries, str):
        queries = [queries]
    queries = list(queries)

    # build compiled patterns
    flags = 0 if case_sensitive else re.IGNORECASE
    patterns: List[re.Pattern] = []
    for q in queries:
        if use_regex:
            pat = re.compile(q, flags)
        else:
            if whole_word:
                # word boundaries around the query; allow gene-like tokens (letters/digits/_-)
                pat = re.compile(rf"\b{re.escape(q)}\b", flags)
            else:
                pat = re.compile(re.escape(q), flags)
        patterns.append(pat)

    # iterate sheets
    xls = pd.ExcelFile(xlsx_path)
    all_hits = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, dtype=str)  # read as strings for robust matching
        if df.empty:
            continue

        # restrict columns if requested (silently ignore missing)
        search_cols = list(columns) if columns is not None else list(df.columns)
        search_cols = [c for c in search_cols if c in df.columns]
        if not search_cols:
            continue

        # pre-clean values (trim whitespace)
        sub = df[search_cols].apply(lambda s: s.str.strip())

        for q, pat in zip(queries, patterns):
            # boolean mask: any column contains the pattern
            col_hits = {c: sub[c].str.contains(pat, na=False, regex=True) for c in search_cols}
            any_hit_mask = pd.DataFrame(col_hits).any(axis=1)

            if not any_hit_mask.any():
                continue

            hits = df.loc[any_hit_mask].copy()
            hits["_sheet"] = sheet
            hits["_row"] = hits.index
            hits["_query"] = q
            # record which columns matched for each row
            match_cols = []
            for idx in hits.index:
                cols = [c for c in search_cols if bool(col_hits[c].loc[idx])]
                match_cols.append(",".join(cols))
            hits["_match_cols"] = match_cols
            all_hits.append(hits)

            if return_first_only:
                # keep only the first matching row in this sheet for this query
                break

    if not all_hits:
        return pd.DataFrame(columns=["_sheet","_row","_query","_match_cols"])

    out = pd.concat(all_hits, ignore_index=True)

    # move helper columns to the front
    helper_cols = ["_sheet","_row","_query","_match_cols"]
    out = out[[*helper_cols, *[c for c in out.columns if c not in helper_cols]]]
    return out

# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
# hits = search_workbook("my_workbook.xlsx", queries="LDHA")

# 2) multiple genes (LDHA + LDHB), only search specific columns
# hits = search_workbook("my_workbook.xlsx", queries=["LDHA","LDHB"], columns=["Gene", "Symbol"])

# 3) save results
# hits.to_excel("matches_LDHA.xlsx", index=False)

# 4) regex search (e.g., LDH(A|B) exactly)
# hits = search_workbook("my_workbook.xlsx", queries=r"^LDH[AB]$", use_regex=True, whole_word=False)

# 5) show which sheet(s) contained LDHA
# sheets_with_ldha = hits.loc[hits["_query"]=="LDHA","_sheet"].unique()
# print(sheets_with_ldha)

In [4]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Ldha")

# 2) multiple genes (LDHA + LDHB), only search specific columns
# hits = search_workbook("my_workbook.xlsx", queries=["LDHA","LDHB"], columns=["Gene", "Symbol"])

# 3) save results
# hits.to_excel("matches_LDHA.xlsx", index=False)

# 4) regex search (e.g., LDH(A|B) exactly)
# hits = search_workbook("my_workbook.xlsx", queries=r"^LDH[AB]$", use_regex=True, whole_word=False)

# 5) show which sheet(s) contained LDHA
# sheets_with_ldha = hits.loc[hits["_query"]=="LDHA","_sheet"].unique()
# print(sheets_with_ldha)

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Marker genes/promoters differentially accessible in each subOLG population,"Pseudobulk aggregation of the expression of differentially expressed genes in MOL56 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the expression of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL56 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed",Average gene regulatory chromatin score of differentially expressed genes in MOL56 between timepoints,Average gene regulatory chromatin score of differentially expressed genes in MOL2 between timepoints
19,Tab3-DGE_GRC_MOL2,345,Ldha,Average gene regulatory chromatin score of dif...,NaN,0.0993006574139845,0.16220732431551,0.206144579115211,0.137785811986747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ldha


In [ ]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Ldha = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Ldha")


In [13]:
hits_Ldha

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Marker genes/promoters differentially accessible in each subOLG population,"Pseudobulk aggregation of the expression of differentially expressed genes in MOL56 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the expression of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL56 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed",Average gene regulatory chromatin score of differentially expressed genes in MOL56 between timepoints,Average gene regulatory chromatin score of differentially expressed genes in MOL2 between timepoints
0,Tab2-DGE_subOLG,800,Ldha,Unnamed: 6,0,2.05857388448025,0.316,0.065,0,OPC-α,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tab2-DGE_subOLG,4099,Ldha,Unnamed: 6,4.98034015699615e-121,2.25970481325952,0.426,0.069,1.60790281968621e-116,OPC-β,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tab2-DGE_subOLG,7566,Ldha,Unnamed: 6,0,2.57194392330653,0.427,0.066,0,OPC-γ,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tab2-DGE_subOLG,11029,Ldha,Unnamed: 6,3.49844274468997e-134,1.56179349527516,0.272,0.068,1.12947224012316e-129,COP,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tab2-DGE_subOLG,13903,Ldha,Unnamed: 6,6.58080673500653e-07,0.865973326817555,0.171,0.069,0.0212461345439686,MOL56-η,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tab2-DGE_subOLG,15173,Ldha,Unnamed: 6,0,2.07729253650233,0.181,0.059,0,MOL56-κ,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tab2-DGE_subOLG,19844,Ldha,Unnamed: 6,5.41385843689784e-89,0.875087486938761,0.124,0.065,1.74786419635247e-84,MOL2-ε,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tab2-DGE_subOLG,20899,Ldha,Unnamed: 6,6.4777224021437e-32,2.19303669620552,0.194,0.069,2.09133267753209e-27,MOL1,Ldha,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Tab2-DGA_subOLG,1230,Ldha,Unnamed: 6,NaN,1.59775773499821,0.33,0.145,2.58735072522455e-131,OPC-α,Ldha,9.98591557400443e-136,NaN,NaN,NaN,NaN,NaN,NaN
9,Tab2-DGA_subOLG,4800,Ldha,Unnamed: 6,NaN,1.78244030623465,0.363,0.148,2.1936208537063e-21,OPC-β,Ldha,8.46630974027905e-26,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Ldhb = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Ldhb")


In [12]:
hits_Ldhb

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Marker genes/promoters differentially accessible in each subOLG population,Marker genes with differential average DORC scores in each subOLG population,"Pseudobulk aggregation of the expression of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed",Average gene regulatory chromatin score of differentially expressed genes in MOL2 between timepoints,Genes differentially expressed between MOL2 vs. MOL56 at each timepoint
0,Tab2-DGE_subOLG,12721,Ldhb,Unnamed: 6,4.46686627532413e-301,1.01338017707491,0.489,0.279,1.44212777698839e-296,MOL56-γ,Ldhb,NaN,NaN,NaN,NaN,NaN,NaN
1,Tab2-DGE_subOLG,13840,Ldhb,Unnamed: 6,9.06768861565443e-11,0.822593542357518,0.524,0.289,2.92750326956403e-06,MOL56-η,Ldhb,NaN,NaN,NaN,NaN,NaN,NaN
2,Tab2-DGE_subOLG,15960,Ldhb,Unnamed: 6,0,1.91287834590951,0.638,0.254,0,MOL2-α,Ldhb,NaN,NaN,NaN,NaN,NaN,NaN
3,Tab2-DGE_subOLG,17905,Ldhb,Unnamed: 6,0,0.730718095425351,0.59,0.273,0,MOL2-δ,Ldhb,NaN,NaN,NaN,NaN,NaN,NaN
4,Tab2-DGE_subOLG,20676,Ldhb,Unnamed: 6,3.74456860773683e-65,2.01765180632838,0.552,0.288,1.20893397500784e-60,MOL1,Ldhb,NaN,NaN,NaN,NaN,NaN,NaN
5,Tab2-DGA_subOLG,3127,Ldhb,Unnamed: 6,NaN,0.696262977914919,0.302,0.229,1.39396536707159e-12,OPC-α,Ldhb,5.38002843331374e-17,NaN,NaN,NaN,NaN,NaN
6,Tab2-DGA_subOLG,7521,Ldhb,Unnamed: 6,NaN,0.672137280805897,0.291,0.23,1,OPC-β,Ldhb,0.0072848817549782,NaN,NaN,NaN,NaN,NaN
7,Tab2-DGA_subOLG,11516,Ldhb,Unnamed: 6,NaN,0.76466175411987,0.271,0.23,1,OPC-γ,Ldhb,0.000268994719750363,NaN,NaN,NaN,NaN,NaN
8,Tab2-DGA_subOLG,17310,Ldhb,Unnamed: 6,NaN,0.948341126084442,0.431,0.21,0,MOL2-α,Ldhb,0,NaN,NaN,NaN,NaN,NaN
9,Tab2-DGA_subOLG,18746,Ldhb,Unnamed: 6,NaN,0.970325336336131,0.436,0.219,0,MOL2-δ,Ldhb,0,NaN,NaN,NaN,NaN,NaN


In [19]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Slc16a1 = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Slc16a1")
hits_Slc16a1

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Marker genes with differential average DORC scores in each subOLG population,"Pseudobulk aggregation of the expression of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed","Pseudobulk aggregation of the accessibility of differentially expressed genes in MOL2 between timepoints, normalized by the total number of counts multiplied by 10,000 and natural log transformed",Average gene regulatory chromatin score of differentially expressed genes in MOL2 between timepoints
0,Tab2-DGE_subOLG,12558,Slc16a1,Unnamed: 6,0,0.672699464015324,0.454,0.388,0,MOL56-β,Slc16a1,NaN,NaN,NaN,NaN
1,Tab2-DGE_subOLG,12738,Slc16a1,Unnamed: 6,1.76839319425192e-220,0.735497739586155,0.591,0.4,5.70925742764233e-216,MOL56-γ,Slc16a1,NaN,NaN,NaN,NaN
2,Tab2-DGE_subOLG,16092,Slc16a1,Unnamed: 6,0,0.533229913285312,0.597,0.39,0,MOL2-α,Slc16a1,NaN,NaN,NaN,NaN
3,Tab2-DVDORCs_subOLG,3037,Slc16a1,Unnamed: 6,NaN,0.753698536568035,0.924,0.918,2.69982796543141e-49,MOL56-α,Slc16a1,8.01373691134285e-53,NaN,NaN,NaN
4,Tab2-DVDORCs_subOLG,3653,Slc16a1,Unnamed: 6,NaN,0.572081776524701,0.916,0.919,3.30201005379888e-159,MOL56-β,Slc16a1,9.80115777322316e-163,NaN,NaN,NaN
5,Tab3-DGE_RNA_MOL2,664,Slc16a1,Pseudobulk aggregation of the expression of di...,NaN,0.967394586549655,0.581893362802595,0.43058346534489,0.575268955502452,NaN,NaN,NaN,Slc16a1,NaN,NaN
6,Tab3-DGE_ACC_MOL2,583,Slc16a1,Pseudobulk aggregation of the accessibility of...,NaN,0.613900381839862,0.581415795111707,0.529230688860463,0.553049454280981,NaN,NaN,NaN,NaN,Slc16a1,NaN
7,Tab3-DGE_GRC_MOL2,561,Slc16a1,Average gene regulatory chromatin score of dif...,NaN,2.08778953787301,1.92954601822968,1.71952300316431,1.86735089738952,NaN,NaN,NaN,NaN,NaN,Slc16a1


In [23]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Hif1a = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Hif1a")
hits_Hif1a

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Tab2-DGE_subOLG,2322,Hif1a,Unnamed: 6,2.12783340186749e-58,0.592175166316989,0.505,0.322,6.86971013792921e-54,OPC-α,Hif1a
1,Tab2-DGE_subOLG,9672,Hif1a,Unnamed: 6,2.85326866692086e-31,0.615316916634019,0.514,0.324,9.211777891154e-27,OPC-γ,Hif1a
2,Tab2-DGE_subOLG,11180,Hif1a,Unnamed: 6,1.79438325659496e-103,1.06034248213972,0.655,0.323,5.79316634391683e-99,COP,Hif1a
3,Tab2-DGE_subOLG,21489,Hif1a,Unnamed: 6,2.50500273032672e-07,1.06702969704813,0.384,0.325,0.00808740131485981,MOL1,Hif1a


In [22]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Ppargc1a = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Ppargc1a")
hits_Ppargc1a

,_sheet,_row,_query,_match_cols,"La sélection que vous tentez de copier-coller est trop grande. Sélectionnez moins de données, puis réessayez.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Marker genes/promoters differentially accessible in each subOLG population,Marker genes with differential average DORC scores in each subOLG population
0,Tab2-DGE_subOLG,1201,Ppargc1a,Unnamed: 6,0,4.89726191863239,0.131,0.004,0,OPC-α,Ppargc1a,NaN,NaN
1,Tab2-DGE_subOLG,13601,Ppargc1a,Unnamed: 6,3.47330254617341e-185,4.50013325968023,0.189,0.006,1.12135572703209e-180,MOL56-η,Ppargc1a,NaN,NaN
2,Tab2-DGA_subOLG,722,Ppargc1a,Unnamed: 6,NaN,1.9899533088568,0.346,0.117,5.64698225399878e-240,OPC-α,Ppargc1a,2.17946053801574e-244,NaN
3,Tab2-DGA_subOLG,4894,Ppargc1a,Unnamed: 6,NaN,1.6372384870572,0.311,0.121,2.468224599919e-19,OPC-β,Ppargc1a,9.52614666120803e-24,NaN
4,Tab2-DGA_subOLG,8449,Ppargc1a,Unnamed: 6,NaN,1.80125844346485,0.337,0.119,1.65703163573133e-107,OPC-γ,Ppargc1a,6.39533630154896e-112,NaN
5,Tab2-DGA_subOLG,13583,Ppargc1a,Unnamed: 6,NaN,1.15739898735271,0.25,0.12,1.76839990002277e-32,COP,Ppargc1a,6.82516364346882e-37,NaN
6,Tab2-DGA_subOLG,18836,Ppargc1a,Unnamed: 6,NaN,1.21522656311617,0.298,0.112,0,MOL2-δ,Ppargc1a,0,NaN
7,Tab2-DGA_subOLG,22148,Ppargc1a,Unnamed: 6,NaN,2.62263105978831,0.408,0.12,2.74531250237832e-99,MOL1,Ppargc1a,1.05955712172069e-103,NaN
8,Tab2-DVDORCs_subOLG,285,Ppargc1a,Unnamed: 6,NaN,0.510496736898928,0.374,0.069,0,OPC-α,Ppargc1a,NaN,0


In [ ]:
# ----- Example usage -----
# 1) simple: find LDHA anywhere, whole-word, case-insensitive
hits_Ppargc1a = search_workbook('/Users/christoffer/Downloads/Supplementary Table.xlsx', queries="Ppargc1a")
hits_Ppargc1a